In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import torch

In [ ]:
!pip install google-cloud-translate

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 198 kB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 118.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 2.3 MB/s s eta 0:00:01
     |████████████████████████████████| 212 kB 118.9 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 102.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 104.3 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 112.3 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 828 kB/s s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install sentencepiece


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.3 MB 17.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 661 kB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 116.0 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 919 kB/s s eta 0:00:01
     |████████████████████████████████| 345 kB 105.8 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 5.6 MB/s s eta 0:00:01
     |████████████████████████████████| 2.0 MB 80.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.9 MB/s s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
data = pd.read_csv("Full_Dataset.csv", header = None)

In [ ]:
data = data.iloc[1:, :4].reset_index(drop=True)

In [ ]:
data

,0,1,2,3
0,هل شيراز هي اللي طلعت كل الـ\nghosts\nوالـ\nmo...,Did Shiraz\nطلعت كل الأشباح والمسوخ دي في\nour...,هل شيراز هي اللي طلعت كل الأشباح والمسوخ دي في...,Did Shiraz put all those ghosts and monsters i...
1,هل ده معناه إن\nShiraz\nحتفضل ورانا\none by on...,Does this mean\nشيراز حتفضل ورانا واحد واحد؟,هل ده معناه إن شيراز حتفضل ورانا واحد واحد؟,Does this mean Shiraz is going to haunt us dow...
2,الأهم من كل ده، الـ\nmost critical question\nإ...,And the most critical question is\n، إزاي ممكن...,الأهم من كل ده، إيه اللي هيوقفها؟,"And the most critical question is, how can she..."
3,بسبب\nrecommendation\nحضرتك الـ\nresearch\nبتا...,Because of\nتوصية حضرتك،\nthe university accep...,بسبب توصية حضرتك البحث بتاعي اتقبل هنا في الكلية,"Because of your recommendation, the university..."
4,يا دكتور هو أنا يعني لاحق أذاكر الـ\nfacts\nلم...,"Doctor, I barely have\nوقت أذاكر الـ واقع وكما...",يا دكتور هو أنا يعني لاحق أذاكر الواقع لما كما...,"Doctor, I barely have enough time to study fac..."
...,...,...,...,...
5222,قلة النوم ممكن يؤدي لـ\nhallucinations\nسمعية ...,Sleep deprivation\nممكن يؤدي لـ هلاوس سمعية وب...,قلة النوم قد تؤدي إلى هلاوس سمعية وبصرية واضطر...,Sleep deprivation can lead to visual and audit...
5223,أنت محتاج الـ\nequipment\nدي كلها\nfor what\nي...,What do you need\nالحاجات دي كلها\nfor\nيا دكتور?,أنت محتاج الحاجات دي كلاتها في إيه يا دكتور؟,"What do you need all this equipment for, doctor?"
5224,في في الشقة\nmouse\nبس كبير شوية.,There's a\nفار\nin my apartment\nبس\nhuge\nشوية.,في في الشقة فار بس كبير شوية,There's a giant mouse in my apartment.
5225,لو عقلك\ntried to trick you\n، خده على قد عقله.,If your mind\nلاعبك،\njust\nخده على قد عقله.,لو عقلك لاعبك، خده على قد عقله,"If your mind tries to trick you, humor it"


In [ ]:
# Loads Phi 3.5 model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

# Define the model pipeline for text generation
pipe = pipeline("text-generation", model="microsoft/Phi-3.5-mini-instruct", device=0, trust_remote_code=True)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


In [ ]:


def translate_phi(arabic_eng_code_switched, english_arab_code_switched):
    # Clean up newlines
    arabic_eng_code_switched = arabic_eng_code_switched.replace("\n", " ").strip()
    english_arab_code_switched = english_arab_code_switched.replace("\n", " ").strip()




    prompt = f"""You are a professional translator.

Task:
1. Translate Input 1 from code-switched Arabic-English to fluent English.
2. Translate Input 2 from code-switched English-Arabic to fluent Egyptian Arabic.

Only return the two translations, one per line, in the same order.

Example Input 1: هو was talking too fast
Example Input 2: I told ه مش هينفع كده


Only return the two translations, one per line, in the same order.
Example Output:
He was talking too fast.
قلتله مش هينفع كده

Now translate the following:

Input 1: {arabic_eng_code_switched}
Input 2: {english_arab_code_switched}
"""

    response = pipe(prompt, max_new_tokens=50)[0]["generated_text"]

    # Extract only the generated translation
    translated_output = response[len(prompt):].strip()
    blacklist_lines = {
    "Only return the two translations, one per line, in the same order.",
    "only return the two translations, one per line, in the same order.",
}



    output_cleaned = translated_output.replace("Output:", "").replace("output:", "").strip()
    lines = output_cleaned.split("\n")

    lines = [line for line in lines if line not in blacklist_lines]
    if lines and lines[0].strip() == "":
        lines = lines[1:]


    #print(lines)
    translation_en = lines[0].strip() if len(lines) > 0 else ""
    translation_ar = lines[1].strip() if len(lines) > 1 else ""
#     print(translation_en)
#     print("")
#     print(translation_ar)
#     print("")
    return translation_en, translation_ar



In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import torch

# Load the model and tokenizer

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

model = MBartForConditionalGeneration.from_pretrained(model_name)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Helper function to translate with mBART
def mbart_translate(text, src_lang_code, tgt_lang_code):
    tokenizer.src_lang = src_lang_code
    encoded = tokenizer(text, return_tensors="pt").to(model.device)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_code],
        max_length=100
    )
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


In [ ]:
def translate_mbart(arabic_eng_code_switched, english_arab_code_switched):
    # Clean inputs
    arabic_eng_code_switched = arabic_eng_code_switched.replace("\n", " ").strip()
    english_arab_code_switched = english_arab_code_switched.replace("\n", " ").strip()

    # Translate Arabic-English to English
    translation_en = mbart_translate(arabic_eng_code_switched, src_lang_code="ar_AR", tgt_lang_code="en_XX")

    # Translate English-Arabic to Egyptian Arabic (approximate with Arabic)
    translation_ar = mbart_translate(english_arab_code_switched, src_lang_code="en_XX", tgt_lang_code="ar_AR")

    #print(f"{translation_en}\n{translation_ar}")

    return translation_en, translation_ar


In [ ]:
from openai import OpenAI
import os


client = OpenAI(api_key="YOUR_API_KEY_HERE" )

def translate_gpt4_1(arabic_eng_code_switched, english_arab_code_switched):
    # Clean up newlines
    arabic_eng_code_switched = arabic_eng_code_switched.replace("\n", " ").strip()
    english_arab_code_switched = english_arab_code_switched.replace("\n", " ").strip()

    prompt = f"""
Translate the following:

1. From Arabic-English code-switching to fluent English:
{arabic_eng_code_switched}

2. From English-Arabic code-switching to fluent Egyptian Arabic:
{english_arab_code_switched}

Return only the two translations, one per line, without out the number 1. or 2 in front and in the same order.
"""

    response = client.chat.completions.create(
        model="gpt-4.1",  # GPT-4.1 model
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=150
    )

    translated_output = response.choices[0].message.content.strip()

    # Clean and filter lines
    blacklist_lines = {
        "Only return the two translations, one per line, in the same order.",
        "only return the two translations, one per line, in the same order.",
        "Output:",
        "output:"
    }

    lines = translated_output.split("\n")
    lines = [line.strip() for line in lines if line.strip() and line.strip() not in blacklist_lines]

    if lines and lines[0].strip() == "":
        lines = lines[1:]


    translation_en = lines[0] if len(lines) > 0 else ""
    translation_ar = lines[1] if len(lines) > 1 else ""
#     print(translation_en, "\n", translation_ar)

    return translation_en, translation_ar

In [ ]:
from google.cloud import translate_v2 as translate
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "feisty-filament-457915-q7-ab86e3b0c79b.json"

# Initialize the Google Cloud Translate client
translate_client = translate.Client()

# Translate text function
def cloud_translate(text, source_lang, target_lang):
    result = translate_client.translate(
        text,
        source_language=source_lang,
        target_language=target_lang
    )
    return result["translatedText"]

# Code-switched translation
def translate_cloud(arabic_eng_code_switched, english_arab_code_switched):
    arabic_eng_code_switched = arabic_eng_code_switched.replace("\n", " ").strip()
    english_arab_code_switched = english_arab_code_switched.replace("\n", " ").strip()

    translation_en = cloud_translate(arabic_eng_code_switched, source_lang="ar", target_lang="en")
    translation_ar = cloud_translate(english_arab_code_switched, source_lang="en", target_lang="ar")

    #print(translation_en, "\n", translation_ar, "\n")

    return translation_en, translation_ar


In [ ]:
#mBART Translations
from tqdm import tqdm

import time

english_translations = []
arabic_translations = []


start_time = time.time() #start time

for i in tqdm(range(len(data[0])), desc="Translating"):
    en, ar = translate_mbart(data[0][i], data[1][i])
    english_translations.append(en)
    arabic_translations.append(ar)

end_time = time.time()  #ends timer
total_time = end_time - start_time

# Print duration in minutes and seconds
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"\nTranslation completed in {minutes}m {seconds}s.")

Translating: 100%|██████████| 5227/5227 [29:08<00:00,  2.99it/s]


✅ Translation completed in 29m 8s.


In [ ]:
#Microsoft Phi 3.5 mini trasnlations
from tqdm import tqdm

import time

english_translations = []
arabic_translations = []


start_time = time.time() #start time

for i in tqdm(range(len(data[0])), desc="Translating"):
    en, ar = translate_phi(data[0][i], data[1][i])
    english_translations.append(en)
    arabic_translations.append(ar)

end_time = time.time()  #ends timer
total_time = end_time - start_time

# Print duration in minutes and seconds
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"\nTranslation completed in {minutes}m {seconds}s.")

Translating: 100%|██████████| 5227/5227 [2:38:10<00:00,  1.82s/it]  


Translation completed in 158m 10s.


In [ ]:
#Gpt 4.1 translations
from tqdm import tqdm

import time

english_translations = []
arabic_translations = []


start_time = time.time() #start time

for i in tqdm(range(len(data[0])), desc="Translating"):
    en, ar = translate_gpt4_1(data[0][i], data[1][i])
    english_translations.append(en)
    arabic_translations.append(ar)

end_time = time.time()  #ends timer
total_time = end_time - start_time

# Print duration in minutes and seconds
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"\nTranslation completed in {minutes}m {seconds}s.")

Translating: 100%|██████████| 5227/5227 [1:36:53<00:00,  1.11s/it]  


Translation completed in 96m 53s.


In [ ]:
#Google Cloud Translate translations
from tqdm import tqdm

import time

english_translations = []
arabic_translations = []


start_time = time.time() #start time

for i in tqdm(range(len(data[0])), desc="Translating"):
    en, ar = translate_cloud(data[0][i], data[1][i])
    english_translations.append(en)
    arabic_translations.append(ar)

end_time = time.time()  #ends timer
total_time = end_time - start_time

# Print duration in minutes and seconds
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"\nTranslation completed in {minutes}m {seconds}s.")

Translating: 100%|██████████| 5227/5227 [19:42<00:00,  4.42it/s]


Translation completed in 19m 42s.


In [ ]:
results = pd.DataFrame({
    "CodeMixed_Arabic_Full_English_Translation": english_translations,
    "CodeMixed_English_Full_Arabic_Translation": arabic_translations
})

# Step 2: Save to CSV
output_csv_path = "oLMo-7B_translate_outputs.csv"
results.to_csv(output_csv_path, index=False)

print(f"Translations saved to: {output_csv_path}")

Translations saved to: google_translate_outputs.csv


In [ ]:
len(english_translations)

0

In [ ]:
english_translations

[]

In [ ]:
arabic_translations

['هل شيراز طلعت كل الأشباح والمسوخ دي على طريقتنا لكي سيبناها؟',
 'هل هذا يعني أن شيراز حتفضل وارانا واحدا واحدا؟',
 'والسؤال الأهم ، إزاي ممكن نـ أوقفها ؟',
 'بسبب مواصفات حضرتك، قبلت الجامعة البحث بتاعي.',
 'دكتور أنا بالكاد أملك وقت أذاكر الـ الحقيقي كمان أذاكر الأساطير.',
 'الآن، انظر، في الأشياء التي يمكنك القيام بها وهي تفتكر أحلامك.',
 'استخدم المفكرة واكتب فيها كل حاجة تفتكرها عن أحلامك، خاصة لو صحيت فجأة في نص الليل.',
 'هل هناك طريقة إني أحلم بـ شخص معين؟',
 'بمقارنة ما كتبته عن الحلم بالـ الأحداث اللي حصل في نفس اليوم.',
 'لذلك بمجرد أن تتمكن من تفك شفرة لاوعيك هتقدر بكل حلم باللي أنت تريد وبالناس اللي نفسك تشفهم.',
 'أنت فقط على مسافة سيجارة واحدة من الموت.',
 'سمعت إنك كنت في جامعة النهاردة. هل هذا صحيح فعلا يا رفعت؟',
 'كان هناك سليم، تقليم المتجر على صدري.',
 'كنت أقوم بتشغيل وحسيت إن الـ الضغط على صدري بيزيد حتى وصلت للمفتاح ولوقتت.',
 'كل شيء في حياتي كان نتيجة لما إنت مرجعتي فيها تاني.',
 'أرى الآن إنك كان لديك وجهة نظر لما قلت إننا مابنفعش لا نبقى مع بعض.',
 'لقد افت

In [ ]:
len(arabic_translations)

5227